#### Libraries

In [105]:
import re, string
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import string

from pprint import pprint

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet

from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans

from nltk import pos_tag, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.decomposition import TruncatedSVD, NMF

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim import corpora, models
from nltk.stem.porter import *

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Busss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Busss\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Loading the data

In [106]:
df = pd.read_csv('data/data.csv')
df.shape

C:\Users\Busss\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(204135, 12)

In [107]:
df.head()

,id,title,author,date,content,year,month,publication,category,digital,section,url
0,1,Agent Cooper in Twin Peaks is the audience: on...,\nTasha Robinson\n,2017-05-31,And never more so than in Showtime’s new...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
1,2,"AI, the humanity!",\nSam Byford\n,2017-05-30,AlphaGo’s victory isn’t a defeat for hum...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
2,3,The Viral Machine,\nKaitlyn Tiffany\n,2017-05-25,Super Deluxe built a weird internet empi...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
3,4,How Anker is beating Apple and Samsung at thei...,\nNick Statt\n,2017-05-22,Steven Yang quit his job at Google in th...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
4,5,Tour Black Panther’s reimagined homeland with ...,\nKwame Opam\n,2017-05-15,Ahead of Black Panther’s 2018 theatrical...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN


#### Removing all columns and keeping only the 'content' column

In [108]:
remove = [i for i in df.columns if i!='title']

In [109]:
df.drop(remove, axis = 1, inplace=True)
df.head()

,title
0,Agent Cooper in Twin Peaks is the audience: on...
1,"AI, the humanity!"
2,The Viral Machine
3,How Anker is beating Apple and Samsung at thei...
4,Tour Black Panther’s reimagined homeland with ...


In [110]:
df.shape

(204135, 1)

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204135 entries, 0 to 204134
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   204132 non-null  object
dtypes: object(1)
memory usage: 1.6+ MB


In [112]:
df.dropna(inplace=True)

In [113]:
df.shape

(204132, 1)

#### Text preprocess functions

In [114]:
stemmer = SnowballStemmer("english")

def lem_stem(word):
    return stemmer.stem(WordNetLemmatizer().lemmatize(word))

def preprocess(text):
    return [lem_stem(token) for token in gensim.utils.simple_preprocess(text) if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3]            


#### Checking the functions at work

In [115]:
print('original:',df['title'].iloc[3][:100])
print("after:",preprocess(df['title'].iloc[3][:100]))

original: How Anker is beating Apple and Samsung at their own accessory game
after: ['anker', 'beat', 'appl', 'samsung', 'accessori', 'game']


#### Preprocessing the data

In [25]:
pp_data = df['title'].map(preprocess)

In [116]:
pp_data[:10]

0    [agent, cooper, twin, peak, audienc, delight, ...
1                                              [human]
2                                      [viral, machin]
3        [anker, beat, appl, samsung, accessori, game]
4    [tour, black, panther, reimagin, homeland, neh...
5                                     [instant, recal]
6                                     [massiv, attack]
7                                       [futur, agenc]
8                                       [flight, risk]
9                                       [brain, drain]
Name: title, dtype: object

#### Bag-of-Words

In [117]:
dict_of_words = gensim.corpora.Dictionary(pp_data)

In [118]:
for i, (key, value) in enumerate(dict_of_words.iteritems()):
    print(key, value)
    if i == 10:
        break

0 agent
1 audienc
2 cooper
3 delight
4 disintegr
5 peak
6 twin
7 human
8 machin
9 viral
10 accessori


#### Total number of words in dict:

In [29]:
print('total:',len(dict_of_words.iteritems()))

total: 32165


#### Filter out tokens
remove < 'no_below', 
remove > 'no_above',
then keep only the first 'keep_n' most

In [119]:
dict_of_words.filter_extremes(no_below=15, no_above=0.1, keep_n=50000)

In [120]:
bow_for_docs = [dict_of_words.doc2bow(i) for i in pp_data]
bow_for_docs[:10]

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(6, 1)],
 [(7, 1), (8, 1)],
 [(9, 1), (10, 1), (11, 1), (12, 1), (13, 1)],
 [(14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)],
 [(21, 1), (22, 1)],
 [(23, 1), (24, 1)],
 [(25, 1), (26, 1)],
 [(27, 1), (28, 1)],
 [(29, 1), (30, 1)]]

#### How many words in the title at the i-th location
(i==2 in this case)

In [32]:
len(bow_for_docs[3])

5

#### Check the words at that location
(id, count)

##### Ordered by how they appear in the content - top 10

In [33]:
bow_for_docs[3]

[(9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]

##### Ordered by the most frequent - top 10

In [34]:
sorted(bow_for_docs[3],  key=lambda x: x[1], reverse=True)[:10]

[(9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]

##### Ordered by the least frequent - top 10

In [35]:
sorted(bow_for_docs[3],  key=lambda x: x[1])[:10]

[(9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]

#### Check the actual words

In [123]:
limit = 10
space = '    '
tmp_words = bow_for_docs[3]

print(f'LOC | WORD NUMBER |  WORD FREQ  | THE ACTUAL WORD')
print('--------------------------------------------------')
for i in range(len(tmp_words)):
    num = str(tmp_words[i][0])
    while len(num) < 3: num = ' '+num
        
    freq = str(tmp_words[i][1])
    while len(freq) < 3: freq = ' '+freq

    loc = str(i)
    while len(loc) < 3: loc = ' '+loc

    print(f'{loc} | {space}{num}{space} | {space}{freq}{space} | {space}{dict_of_words[tmp_words[i][0]]}')
    if i == limit: break
print(tmp_words)

LOC | WORD NUMBER |  WORD FREQ  | THE ACTUAL WORD
--------------------------------------------------
  0 |       9     |       1     |     accessori
  1 |      10     |       1     |     appl
  2 |      11     |       1     |     beat
  3 |      12     |       1     |     game
  4 |      13     |       1     |     samsung
[(9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]


### TF-IDF

In [86]:
idf = models.TfidfModel(bow_for_docs)

##### Transforming

In [125]:
tdf_data = idf[bow_for_docs]

##### (id, score)

In [88]:
for i in tdf_data:
    print(i)
    break

[(0, 0.3480650151787403), (1, 0.4209161975451784), (2, 0.4047524798643647), (3, 0.46501661778648373), (4, 0.4151525454802232), (5, 0.3863465271748787)]


### LDA
Training

In [89]:
lda = gensim.models.LdaMulticore(bow_for_docs, num_topics=5, id2word=dict_of_words, passes=10, workers=6)

##### Lets view the words

In [90]:
for i, topic in lda.print_topics(-1):
    print(f"Topic: {i} \nWords: {topic}\n---\n")

Topic: 0 
Words: 0.013*"hous" + 0.012*"say" + 0.011*"republican" + 0.010*"breitbart" + 0.010*"white" + 0.010*"obama" + 0.009*"russia" + 0.007*"plan" + 0.007*"health" + 0.007*"senat"
---

Topic: 1 
Words: 0.011*"life" + 0.008*"american" + 0.008*"world" + 0.008*"opinion" + 0.008*"dy" + 0.007*"video" + 0.007*"review" + 0.006*"home" + 0.005*"year" + 0.005*"histori"
---

Topic: 2 
Words: 0.029*"opinion" + 0.013*"america" + 0.012*"breitbart" + 0.011*"north" + 0.010*"korea" + 0.010*"presid" + 0.009*"donald" + 0.009*"woman" + 0.008*"week" + 0.007*"brief"
---

Topic: 3 
Words: 0.015*"kill" + 0.015*"attack" + 0.012*"polic" + 0.011*"court" + 0.009*"state" + 0.008*"case" + 0.008*"death" + 0.008*"offic" + 0.008*"shoot" + 0.007*"say"
---

Topic: 4 
Words: 0.027*"clinton" + 0.015*"donald" + 0.014*"hillari" + 0.010*"like" + 0.010*"breitbart" + 0.007*"look" + 0.007*"year" + 0.007*"wall" + 0.005*"win" + 0.005*"appl"
---



#### Topic Classification

In [91]:
'Topic: 0 Words: 0.015*"player" + 0.012*"season" + 0.009*"sport" + 0.008*"leagu" + 0.007*"coach" + 0.007*"olymp" + 0.006*"athlet" + 0.005*"footbal" + 0.005*"field" + 0.005*"ball"'

'Topic: 0 Words: 0.015*"player" + 0.012*"season" + 0.009*"sport" + 0.008*"leagu" + 0.007*"coach" + 0.007*"olymp" + 0.006*"athlet" + 0.005*"footbal" + 0.005*"field" + 0.005*"ball"'

### LDA + TF-IDF
Training

In [92]:
lda_idf = gensim.models.LdaMulticore(tdf_data, num_topics=5, id2word=dict_of_words, passes=15, workers=6)

In [93]:
for i, topic in lda_idf.print_topics(-1):
    print(f"Topic: {i} \nWords: {topic}\n---\n")

Topic: 0 
Words: 0.004*"home" + 0.004*"opinion" + 0.004*"appl" + 0.004*"wall" + 0.004*"street" + 0.004*"world" + 0.003*"sale" + 0.003*"year" + 0.003*"record" + 0.003*"crash"
---

Topic: 1 
Words: 0.007*"north" + 0.007*"opinion" + 0.006*"korea" + 0.006*"court" + 0.005*"say" + 0.005*"state" + 0.005*"attack" + 0.005*"syria" + 0.004*"china" + 0.004*"deal"
---

Topic: 2 
Words: 0.006*"dy" + 0.005*"year" + 0.005*"woman" + 0.005*"life" + 0.005*"polic" + 0.004*"review" + 0.004*"shoot" + 0.004*"star" + 0.004*"kill" + 0.004*"death"
---

Topic: 3 
Words: 0.012*"clinton" + 0.010*"opinion" + 0.008*"donald" + 0.008*"republican" + 0.007*"hillari" + 0.007*"democrat" + 0.007*"health" + 0.006*"breitbart" + 0.006*"read" + 0.006*"presid"
---

Topic: 4 
Words: 0.018*"opinion" + 0.005*"brief" + 0.005*"book" + 0.004*"news" + 0.004*"america" + 0.004*"even" + 0.004*"facebook" + 0.004*"thing" + 0.004*"olymp" + 0.004*"breitbart"
---



#### Topic Classification

In [94]:
'Topic: 0 Words: 0.010*"olymp" + 0.009*"yanke" + 0.008*"player" + 0.007*"athlet" + 0.007*"met" + 0.006*"cuban" + 0.006*"cuba" + 0.006*"basebal" + 0.006*"leagu" + 0.006*"season"'

'Topic: 0 Words: 0.010*"olymp" + 0.009*"yanke" + 0.008*"player" + 0.007*"athlet" + 0.007*"met" + 0.006*"cuban" + 0.006*"cuba" + 0.006*"basebal" + 0.006*"leagu" + 0.006*"season"'

### Testing
###### Using these parameters

In [95]:
loc = 321
tmp_doc = pp_data[loc]
print(tmp_doc)

['watch', 'histori', 'movi', 'trailer', 'told', 'star', 'war']


###### Using this article 

In [96]:
print('original:',df['title'].iloc[loc])

original: Watch the history of the movie trailer, as told by Star Wars


#### LDA model

##### Lets check which topic 'tmp_doc' above belongs to

In [97]:
for i, conf in sorted(lda[bow_for_docs[loc]], key=lambda x: x[1]*-1):
    print(f"Score: {conf}\t \nTopic: {lda.print_topic(i, 10)}\n")

Score: 0.729004442691803	 
Topic: 0.029*"opinion" + 0.013*"america" + 0.012*"breitbart" + 0.011*"north" + 0.010*"korea" + 0.010*"presid" + 0.009*"donald" + 0.009*"woman" + 0.008*"week" + 0.007*"brief"

Score: 0.1957699954509735	 
Topic: 0.011*"life" + 0.008*"american" + 0.008*"world" + 0.008*"opinion" + 0.008*"dy" + 0.007*"video" + 0.007*"review" + 0.006*"home" + 0.005*"year" + 0.005*"histori"

Score: 0.025080086663365364	 
Topic: 0.015*"kill" + 0.015*"attack" + 0.012*"polic" + 0.011*"court" + 0.009*"state" + 0.008*"case" + 0.008*"death" + 0.008*"offic" + 0.008*"shoot" + 0.007*"say"

Score: 0.025074927136301994	 
Topic: 0.027*"clinton" + 0.015*"donald" + 0.014*"hillari" + 0.010*"like" + 0.010*"breitbart" + 0.007*"look" + 0.007*"year" + 0.007*"wall" + 0.005*"win" + 0.005*"appl"

Score: 0.025070572271943092	 
Topic: 0.013*"hous" + 0.012*"say" + 0.011*"republican" + 0.010*"breitbart" + 0.010*"white" + 0.010*"obama" + 0.009*"russia" + 0.007*"plan" + 0.007*"health" + 0.007*"senat"



###### After viewing the original text and the cluster/classification is 'topic 0' which is movies/arrt/review with 72% confidence the model is correct.

### Testing LDA + TF-IDF model

In [98]:
for i, conf in sorted(lda_idf[bow_for_docs[loc]], key=lambda x: x[1]*-1):
    print(f"Score: {conf}\t \nTopic: {lda_idf.print_topic(i, 10)}\n")

Score: 0.8992657661437988	 
Topic: 0.006*"dy" + 0.005*"year" + 0.005*"woman" + 0.005*"life" + 0.005*"polic" + 0.004*"review" + 0.004*"shoot" + 0.004*"star" + 0.004*"kill" + 0.004*"death"

Score: 0.025354212149977684	 
Topic: 0.018*"opinion" + 0.005*"brief" + 0.005*"book" + 0.004*"news" + 0.004*"america" + 0.004*"even" + 0.004*"facebook" + 0.004*"thing" + 0.004*"olymp" + 0.004*"breitbart"

Score: 0.025177814066410065	 
Topic: 0.004*"home" + 0.004*"opinion" + 0.004*"appl" + 0.004*"wall" + 0.004*"street" + 0.004*"world" + 0.003*"sale" + 0.003*"year" + 0.003*"record" + 0.003*"crash"

Score: 0.025153230875730515	 
Topic: 0.012*"clinton" + 0.010*"opinion" + 0.008*"donald" + 0.008*"republican" + 0.007*"hillari" + 0.007*"democrat" + 0.007*"health" + 0.006*"breitbart" + 0.006*"read" + 0.006*"presid"

Score: 0.02504894696176052	 
Topic: 0.007*"north" + 0.007*"opinion" + 0.006*"korea" + 0.006*"court" + 0.005*"say" + 0.005*"state" + 0.005*"attack" + 0.005*"syria" + 0.004*"china" + 0.004*"deal"



###### After viewing the original and the the cluster/classification. It appears that the model is 90% confident that the topic is 'topic 0' Which is movie/review/life. That is the correct classification for the document.


### Test on new documents
article from: https://www.arabnews.com/node/1992296/sport

In [102]:
new_doc = 'Ghostbusters Learned From Force Awakens Mistakes (And It Worked Perfectly)'
bow = dict_of_words.doc2bow(preprocess(new_doc))

#### Testing LDA on new article

In [103]:
for i, conf in sorted(lda[bow], key=lambda x: x[1]*-1):
    print(f"Score: {conf}\t \nTopic: {lda.print_topic(i, 10)}\n")

Score: 0.5589032769203186	 
Topic: 0.011*"life" + 0.008*"american" + 0.008*"world" + 0.008*"opinion" + 0.008*"dy" + 0.007*"video" + 0.007*"review" + 0.006*"home" + 0.005*"year" + 0.005*"histori"

Score: 0.24325038492679596	 
Topic: 0.013*"hous" + 0.012*"say" + 0.011*"republican" + 0.010*"breitbart" + 0.010*"white" + 0.010*"obama" + 0.009*"russia" + 0.007*"plan" + 0.007*"health" + 0.007*"senat"

Score: 0.14723505079746246	 
Topic: 0.029*"opinion" + 0.013*"america" + 0.012*"breitbart" + 0.011*"north" + 0.010*"korea" + 0.010*"presid" + 0.009*"donald" + 0.009*"woman" + 0.008*"week" + 0.007*"brief"

Score: 0.025336600840091705	 
Topic: 0.015*"kill" + 0.015*"attack" + 0.012*"polic" + 0.011*"court" + 0.009*"state" + 0.008*"case" + 0.008*"death" + 0.008*"offic" + 0.008*"shoot" + 0.007*"say"

Score: 0.02527468465268612	 
Topic: 0.027*"clinton" + 0.015*"donald" + 0.014*"hillari" + 0.010*"like" + 0.010*"breitbart" + 0.007*"look" + 0.007*"year" + 0.007*"wall" + 0.005*"win" + 0.005*"appl"



#### Result
The LDA model has classified the aritcle correctly into review/life/movies. 55%confidence.

#### Testing LDA + TF-IDF on new article

In [104]:
for i, conf in sorted(lda_idf[bow], key=lambda x: x[1]*-1):
    print(f"Score: {conf}\t \nTopic: {lda_idf.print_topic(i, 10)}\n")

Score: 0.4485916495323181	 
Topic: 0.006*"dy" + 0.005*"year" + 0.005*"woman" + 0.005*"life" + 0.005*"polic" + 0.004*"review" + 0.004*"shoot" + 0.004*"star" + 0.004*"kill" + 0.004*"death"

Score: 0.29611530900001526	 
Topic: 0.012*"clinton" + 0.010*"opinion" + 0.008*"donald" + 0.008*"republican" + 0.007*"hillari" + 0.007*"democrat" + 0.007*"health" + 0.006*"breitbart" + 0.006*"read" + 0.006*"presid"

Score: 0.20367766916751862	 
Topic: 0.004*"home" + 0.004*"opinion" + 0.004*"appl" + 0.004*"wall" + 0.004*"street" + 0.004*"world" + 0.003*"sale" + 0.003*"year" + 0.003*"record" + 0.003*"crash"

Score: 0.026074189692735672	 
Topic: 0.007*"north" + 0.007*"opinion" + 0.006*"korea" + 0.006*"court" + 0.005*"say" + 0.005*"state" + 0.005*"attack" + 0.005*"syria" + 0.004*"china" + 0.004*"deal"

Score: 0.025541190057992935	 
Topic: 0.018*"opinion" + 0.005*"brief" + 0.005*"book" + 0.004*"news" + 0.004*"america" + 0.004*"even" + 0.004*"facebook" + 0.004*"thing" + 0.004*"olymp" + 0.004*"breitbart"



#### Result
The LDA+TF-IDF model has classified the aritcle correctly into review/life/movies.(45% confidence)